In [1]:
import pandas as pd
import numpy as np
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy import displacy
import json
import random
import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Read Android reviews

df_orginal_android = pd.read_excel('../data/qb/original/android.xlsx')

In [3]:
len(df_orginal_android)

12281

In [4]:
df_orginal_android.head()

,App,Store,App ID,Review ID,Country,Version,Rating,Date,Author,Subject,...,Translated Subject,Translated Body,Sentiment,Device,Language,OS Version,Reply URL,Topics,Custom Topics,Tags
0,QuickBooks Accounting: Invoicing & Expenses,Google Play,com.intuit.quickbooks,gp:AOqpTOG8sngLHraw-SvWUP7rfDkg-J6U2Rib0x24pYw...,English,19.12.0.12,1,2020-01-31,Salimeh Safi,NaN,...,NaN,NaN,negative,Samsung Galaxy A8(2018) SM-A530W,English,28.0,https://app.appbot.co/apps/1846403-quickbooks-...,"[""Design & UX""]",[],[]
1,QuickBooks Accounting: Invoicing & Expenses,Google Play,com.intuit.quickbooks,gp:AOqpTOFDshU4SGpgBWMiMO8BQid5oDRlP7lgpRWbYpy...,English,20.0.1,1,2020-01-31,Martin K,NaN,...,NaN,NaN,negative,OnePlus 7T HD1905,English,29.0,https://app.appbot.co/apps/1846403-quickbooks-...,"[""Camera & Photos"", ""Pricing"", ""Customer Suppo...","[""Cost"", ""Mileage"", ""Invoicing"", ""Phone Support""]",[]
2,QuickBooks Accounting: Invoicing & Expenses,Google Play,com.intuit.quickbooks,gp:AOqpTOGBuNGfmERh1Mk39ZpTIO7zvUD9wyGjf8mE41z...,English,20.0.1,5,2020-01-31,Kevin Walsh,NaN,...,NaN,NaN,positive,Asus ZenFone 5 Lite (ZC600KL) ASUS_X017DA/ZC600KL,English,28.0,https://app.appbot.co/apps/1846403-quickbooks-...,"[""Satisfied users""]",[],[]
3,QuickBooks Accounting: Invoicing & Expenses,Google Play,com.intuit.quickbooks,gp:AOqpTOFwNiVNulk9JuJ5uhOoJ8SyrSgC0zF1uSnVsvk...,English,19.12.0.12,4,2020-01-31,Kevin Ollivier,NaN,...,NaN,NaN,mixed,Xiaomi Redmi Note 4 Redmi Note 4/Redmi Note 4X,English,24.0,https://app.appbot.co/apps/1846403-quickbooks-...,[],"[""DashboardUpdate""]",[]
4,QuickBooks Accounting: Invoicing & Expenses,Google Play,com.intuit.quickbooks,gp:AOqpTOHeewsms2xOi9BCsk2sSL-EByKf721FtAYdH_b...,English,19.12.0.12,5,2020-01-31,David Cadieux,NaN,...,NaN,NaN,positive,Asus ZenFone 3 Zoom (ZE553KL),English,26.0,https://app.appbot.co/apps/1846403-quickbooks-...,"[""Satisfied users""]",[],[]


In [5]:
#Import IOS reviews
df_orginal_ios = pd.read_excel('../data/qb/original/ios.xlsx')

In [6]:
df_orginal_ios.head()

,App,Store,App ID,Review ID,Country,Version,Rating,Date,Author,Subject,...,Translated Subject,Translated Body,Sentiment,Device,Language,OS Version,Reply URL,Topics,Custom Topics,Tags
0,QuickBooks Accounting,iOS,584606479,5.467988e+09,USA,20.01.6,1,2020-01-31,NanaHermanson,Shiny useless tool,...,NaN,NaN,negative,NaN,English,NaN,https://app.appbot.co/apps/311455-quickbooks-a...,"[""Advertising"", ""Bugs"", ""Design & UX"", ""Dissat...","[""Ease of Use""]",[]
1,QuickBooks Accounting,iOS,584606479,5.464235e+09,Canada,20.01.5,1,2020-01-30,Tugboat55,Latest app update,...,NaN,NaN,negative,NaN,English,NaN,https://app.appbot.co/apps/311455-quickbooks-a...,"[""Bugs"", ""Feature Requests"", ""Dissatisfied use...",[],[]
2,QuickBooks Accounting,iOS,584606479,5.467098e+09,USA,20.01.5,3,2020-01-30,scholarbev,App closes out while entering expenses,...,NaN,NaN,negative,NaN,English,NaN,https://app.appbot.co/apps/311455-quickbooks-a...,"[""Bugs"", ""Frequency"", ""Pricing"", ""Customer Sup...","[""FDX Filter""]",[]
3,QuickBooks Accounting,iOS,584606479,5.466985e+09,USA,20.01.5,5,2020-01-30,SOSA EL DINERO,Life saver,...,NaN,NaN,positive,NaN,English,NaN,https://app.appbot.co/apps/311455-quickbooks-a...,"[""Design & UX"", ""Performance"", ""Satisfied user...",[],[]
4,QuickBooks Accounting,iOS,584606479,5.467691e+09,USA,20.01.6,5,2020-01-30,Sra. Linda,Small business,...,NaN,NaN,positive,NaN,English,NaN,https://app.appbot.co/apps/311455-quickbooks-a...,"[""Satisfied users""]",[],[]


In [7]:
#Combine both reviews

df_orginal = df_orginal_ios.append(df_orginal_android, ignore_index=True)

In [8]:
#Get selective columns

df = df_orginal[['Review ID','Country', 'Version', 'Rating', 'Date', 'Subject', 'Body', 'Sentiment']]

In [9]:
df.head()

,Review ID,Country,Version,Rating,Date,Subject,Body,Sentiment
0,5.46799e+09,USA,20.01.6,1,2020-01-31,Shiny useless tool,This app looks awesome. Very well thought out...,negative
1,5.46424e+09,Canada,20.01.5,1,2020-01-30,Latest app update,This update is terrible basically unusable ple...,negative
2,5.4671e+09,USA,20.01.5,3,2020-01-30,App closes out while entering expenses,The majority of the time this app is wonderful...,negative
3,5.46699e+09,USA,20.01.5,5,2020-01-30,Life saver,One my favorite apps to use while working. I’m...,positive
4,5.46769e+09,USA,20.01.6,5,2020-01-30,Small business,This absolutely works perfectly without diffic...,positive


In [10]:
#replace odd values

df['Subject'] = df['Subject'].replace(np.nan, '', regex=True)
df['Body'] = df['Body'].replace(np.nan, '', regex=True)

In [11]:
df.head()

,Review ID,Country,Version,Rating,Date,Subject,Body,Sentiment
0,5.46799e+09,USA,20.01.6,1,2020-01-31,Shiny useless tool,This app looks awesome. Very well thought out...,negative
1,5.46424e+09,Canada,20.01.5,1,2020-01-30,Latest app update,This update is terrible basically unusable ple...,negative
2,5.4671e+09,USA,20.01.5,3,2020-01-30,App closes out while entering expenses,The majority of the time this app is wonderful...,negative
3,5.46699e+09,USA,20.01.5,5,2020-01-30,Life saver,One my favorite apps to use while working. I’m...,positive
4,5.46769e+09,USA,20.01.6,5,2020-01-30,Small business,This absolutely works perfectly without diffic...,positive


In [12]:
#Total number of reviews

len(df)

19841

In [13]:
#combine subject and body

for index, row in df.iterrows():
    sub = row['Subject']
    body = row['Body']
    
    df.loc[index,'Review'] = str(sub) + ". " + str(body)

In [14]:
df.head()

,Review ID,Country,Version,Rating,Date,Subject,Body,Sentiment,Review
0,5.46799e+09,USA,20.01.6,1,2020-01-31,Shiny useless tool,This app looks awesome. Very well thought out...,negative,Shiny useless tool. This app looks awesome. V...
1,5.46424e+09,Canada,20.01.5,1,2020-01-30,Latest app update,This update is terrible basically unusable ple...,negative,Latest app update. This update is terrible bas...
2,5.4671e+09,USA,20.01.5,3,2020-01-30,App closes out while entering expenses,The majority of the time this app is wonderful...,negative,App closes out while entering expenses. The ma...
3,5.46699e+09,USA,20.01.5,5,2020-01-30,Life saver,One my favorite apps to use while working. I’m...,positive,Life saver. One my favorite apps to use while ...
4,5.46769e+09,USA,20.01.6,5,2020-01-30,Small business,This absolutely works perfectly without diffic...,positive,Small business. This absolutely works perfectl...


In [15]:
#Drop Subject and Body

del df['Subject']
del df['Body']

In [16]:
df.head()

,Review ID,Country,Version,Rating,Date,Sentiment,Review
0,5.46799e+09,USA,20.01.6,1,2020-01-31,negative,Shiny useless tool. This app looks awesome. V...
1,5.46424e+09,Canada,20.01.5,1,2020-01-30,negative,Latest app update. This update is terrible bas...
2,5.4671e+09,USA,20.01.5,3,2020-01-30,negative,App closes out while entering expenses. The ma...
3,5.46699e+09,USA,20.01.5,5,2020-01-30,positive,Life saver. One my favorite apps to use while ...
4,5.46769e+09,USA,20.01.6,5,2020-01-30,positive,Small business. This absolutely works perfectl...


In [18]:
#Get training data (manually generated) for the aspect model 

filename = "../data/qb/processed/train_ner.json"
print(filename)


with open(filename) as train_data:
    train = json.load(train_data)

TRAIN_DATA_NEW = []
for data in train:
    ents = [tuple(entity) for entity in data['entities']]
    TRAIN_DATA_NEW.append((data['content'],{'entities':ents}))


with open('{}'.format(filename.replace('json','txt')),'w') as write:
    write.write(str(TRAIN_DATA_NEW))


../data/qb/processed/train_ner.json


#### Spacy model finds most relevant word(s) to a predefined aspect

In [19]:
#Generate Spacy model for a pre-defiend set of aspects

def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.load('en_core_web_sm')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    else:
        ner = nlp.get_pipe("ner")
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp


prdnlp = train_spacy(TRAIN_DATA_NEW, 20)

Statring iteration 0
{'ner': 2225.028007701685}
Statring iteration 1
{'ner': 1842.3788210980288}
Statring iteration 2
{'ner': 1970.197853528217}
Statring iteration 3
{'ner': 2035.2249178325246}
Statring iteration 4
{'ner': 2021.186934841864}
Statring iteration 5
{'ner': 1956.2226042543807}
Statring iteration 6
{'ner': 1862.7897757522464}
Statring iteration 7
{'ner': 1848.8290638973897}
Statring iteration 8
{'ner': 1891.7265437821075}
Statring iteration 9
{'ner': 1846.6853621557407}
Statring iteration 10
{'ner': 1818.9584692244773}
Statring iteration 11
{'ner': 1810.6771083049975}
Statring iteration 12
{'ner': 1999.9922914089632}
Statring iteration 13
{'ner': 1854.5550557321747}
Statring iteration 14
{'ner': 1824.8694903383725}
Statring iteration 15
{'ner': 1851.5705214914365}
Statring iteration 16
{'ner': 1791.0484162499333}
Statring iteration 17
{'ner': 1848.0194345996692}
Statring iteration 18
{'ner': 1791.2225508622273}
Statring iteration 19
{'ner': 1694.3001642637023}


In [20]:
# Save trained Model
prdnlp.to_disk("./model/qbo_aspect")

In [21]:
#Test the model
test_text = ["Cancelling your subscription seems impossible. I've emailed, called, used their help menu. It shouldn't be this hard. Still trying to cancel.",
              "My App has stopped working on my Samsung s9 for 2 days in a row. You should have sent an email to inform subscribers if your app is going to be experiencing a downtime.",
              "Cannot open customer invoices. Please correct ASAP!",
               "bugs on iOS13 please update ASAP!"]
for text in test_text:
    print('-----')
    doc = prdnlp(text)
    for ent in doc.ents:
        print('keyword=' + ent.text, 'aspect=' + ent.label_)

-----
keyword=subscription aspect=Subscription
keyword=called aspect=Quality
-----
keyword=App aspect=App
-----
keyword=customer invoices aspect=Customer Service
-----
keyword=update aspect=Updates


In [23]:
#For each review, identify and collect one or aspect and corresponding word(s) 

for index, row in df.iterrows():
    aspect_terms = []
    aspect_related_text = []
    doc = prdnlp(row['Review'])
    for ent in doc.ents:
        if ent.label_ not in aspect_terms:            
            aspect_terms.append(ent.label_)
            aspect_related_text.append(str(ent))

    df.loc[index,'Aspects'] = ", ".join(aspect_terms)
    df.loc[index,'AspectText'] = ", ".join(aspect_related_text)

In [24]:
df.head()

,Review ID,Country,Version,Rating,Date,Sentiment,Review,Aspects,AspectText
0,5.46799e+09,USA,20.01.6,1,2020-01-31,negative,Shiny useless tool. This app looks awesome. V...,"Experience, App, Expenses, Customer Service","useless, app, expenses, support"
1,5.46424e+09,Canada,20.01.5,1,2020-01-30,negative,Latest app update. This update is terrible bas...,"App, Updates","app, update"
2,5.4671e+09,USA,20.01.5,3,2020-01-30,negative,App closes out while entering expenses. The ma...,"App, Expenses, Quality","App, expenses, issue"
3,5.46699e+09,USA,20.01.5,5,2020-01-30,positive,Life saver. One my favorite apps to use while ...,App,apps
4,5.46769e+09,USA,20.01.6,5,2020-01-30,positive,Small business. This absolutely works perfectl...,,


In [25]:
#number of reviews without aspect
(df['Aspects'].values == '').sum() 

7666

In [26]:
len(df)

19841

In [27]:
df['Aspects'].replace('', np.nan, inplace=True)
df.dropna(subset=['Aspects'], inplace=True)

In [28]:
len(df)

12175

In [29]:
df.head(50)

,Review ID,Country,Version,Rating,Date,Sentiment,Review,Aspects,AspectText
0,5.46799e+09,USA,20.01.6,1,2020-01-31,negative,Shiny useless tool. This app looks awesome. V...,"Experience, App, Expenses, Customer Service","useless, app, expenses, support"
1,5.46424e+09,Canada,20.01.5,1,2020-01-30,negative,Latest app update. This update is terrible bas...,"App, Updates","app, update"
2,5.4671e+09,USA,20.01.5,3,2020-01-30,negative,App closes out while entering expenses. The ma...,"App, Expenses, Quality","App, expenses, issue"
3,5.46699e+09,USA,20.01.5,5,2020-01-30,positive,Life saver. One my favorite apps to use while ...,App,apps
5,5.46055e+09,USA,20.01.4,1,2020-01-29,negative,Worst update yet. Just deleted. Crashed. Didn’...,"Updates, Quality","update, deleted"
6,5.46148e+09,USA,20.01.4,1,2020-01-29,negative,Transferring to TurboTax. Transferring company...,"Quality, Expenses","issue, customers"
7,5.46253e+09,USA,20.01.4,5,2020-01-29,positive,Great app for small business owners!. I have a...,App,app
8,5.46281e+09,USA,20.01.4,4,2020-01-29,mixed,Good but needs multi currency. Very easy to us...,"Quality, Invoicing, Expenses","US, invoice, customers"
10,5.45735e+09,United Kingdom,20.01.4,4,2020-01-28,neutral,Good App but. It’s a good app but you should b...,"App, Invoicing","App, invoices"
11,5.45882e+09,USA,20.01.4,3,2020-01-28,negative,Having issues with iPhone 11. After installing...,"Quality, App","issues, app"


In [30]:
#generate aspect position index

def generate_df_with_position(data_frame):

    df_split_terms = pd.DataFrame(columns=['Review ID', 
                                           'Country', 'Version', 
                                           'Rating', 'Date', 'Doc Sentiment', 'Asp Sentiment', 
                                           'Review', 'Aspects', 'AspectText', 'Positions'])


    for index, row in data_frame.iterrows():
        aspecttexts = [x.strip() for x in row['AspectText'].lower().split(',')]
        aspects = [x.strip() for x in row['Aspects'].lower().split(',')]

        positions = ''
        for idx, asp in enumerate(aspecttexts):

            review = " ".join(row['Review'].lower().split())

            start_idx = review.find(asp)

            #if the aspect is not found in the review, skip
            if start_idx == -1:
                continue

            sentiment = 'neutral' if row['Sentiment'] == 'mixed' else row['Sentiment']
            
            
            df_split_terms = df_split_terms.append({
                'Review ID': row['Review ID'], 
                'Country': row['Country'], 
                'Version': row['Version'],
                'Rating': row['Rating'],
                'Date': row['Date'],            
                'Doc Sentiment': sentiment,
                'Asp Sentiment' : '' if len(aspecttexts) > 1 else sentiment,
                'Review': review,
                'AspectText': asp,
                'Aspects': aspects[idx] if len(aspects) > idx else asp,
                'Positions': str(start_idx) + ',' + str(start_idx + len(asp) - 1),
            }, ignore_index=True)

    return df_split_terms

In [31]:
df_split_terms = generate_df_with_position(df)
df_split_terms.head()

,Review ID,Country,Version,Rating,Date,Doc Sentiment,Asp Sentiment,Review,Aspects,AspectText,Positions
0,5.46799e+09,USA,20.01.6,1,2020-01-31,negative,,shiny useless tool. this app looks awesome. ve...,experience,useless,"6,12"
1,5.46799e+09,USA,20.01.6,1,2020-01-31,negative,,shiny useless tool. this app looks awesome. ve...,app,app,"25,27"
2,5.46799e+09,USA,20.01.6,1,2020-01-31,negative,,shiny useless tool. this app looks awesome. ve...,expenses,expenses,"190,197"
3,5.46799e+09,USA,20.01.6,1,2020-01-31,negative,,shiny useless tool. this app looks awesome. ve...,customer service,support,"435,441"
4,5.46424e+09,Canada,20.01.5,1,2020-01-30,negative,,latest app update. this update is terrible bas...,app,app,"7,9"


In [32]:
len(df_split_terms)

19656

In [36]:

df_split_terms.to_csv(r'data/qb/processed/combine_aspect_position.csv')